<a href="https://colab.research.google.com/github/Tecindes/ChallengeONE__/blob/main/TelecomX_parte2_latam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

url = "https://raw.githubusercontent.com/alura-cursos/challenge2-data-science-LATAM/main/TelecomX_Data.json"
respuesta = requests.get(url)

if respuesta.status_code == 200:
    dataURL = respuesta.json()
    print(f'Tipo devuelto de la solicitud HTTP GET : {type(dataURL)}')
else:
    print("Error en la conexión")